Import library dan paket yang diperlukan untuk topic modelling (terdiri atas pandas, numpy, re, string, spacy, gensim)

In [1]:
import pandas as pd
import numpy as np

import re
import string

import spacy
import gensim
from gensim import corpora

In [2]:
!pip install pyldavis

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.2 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyldavis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=6ffb557e79d6bcc6218bfe07ee0621e81323037d3dd27b11743dd4f0c1288763
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1310 sha256=2f04dacb28cefee0b618caf2812191526597c5a7f8ae345bea3ee99990da7d48
  Stored in directory: /root/.cache/pip/wheels/46/ef/c3/157e41f5ee1372d1be90b09f74f82b10e391eaacca8f22d33e
Successfully built pyldavis sklearn


Import paket yang diperlukan

In [3]:
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
import os
os.chdir('./drive/My Drive/Colab Notebooks/nlp/UTS/BAGIAN KEDUA')

Membaca isi file Reviews_Filter.csv dan melihat bagian teratas pada isi file tersebut.

In [6]:
review_data = pd.read_csv('Reviews_Filter.csv')
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Membuat fungsi untuk cleaning data

In [7]:
def clean_text(text):
  delete_dict ={sp_character: '' for sp_character in string.punctuation}
  delete_dict[' '] = ' '
  table = str.maketrans(delete_dict)
  text1 = text.translate(table)
  textArr = text1.split()
  text2 = ' '.join([w for w in textArr if (not w.isdigit() and (not w.isdigit() and len(w)>3))])

  return text2.lower()

Download paket stopwords (menghilangkan kakat-kata yang tidak punya makna) pada nltk

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [10]:
review_data.dropna(axis = 0, how = 'any', inplace=True)

In [11]:
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


Melakukan proses cleaning data pada fitur Text, kemudian menambahkan fitur baru dengan nama Num_words_text yang berisi jumlah kata pada fitur Text

In [12]:
review_data['Text'] = review_data['Text'].apply(clean_text)
review_data['Num_words_text'] = review_data['Text'].apply(lambda x:len(str(x).split()))

Melihat hasil teratas fiitur Text yang telah di cleaning

In [13]:
review_data.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,have bought several vitality canned food produ...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this confection that been around centuries lig...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe h...,22
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,great taffy great price there wide assortment ...,17


Membuat variabel baru dengan nama max_review_data_sentence_length yang berisi jumlah kata tertinggi pada fitur Num_words_text. Kemudian membuat variabel baru dengan nama mask dimana isi variabel tersebut berisi kata-kata yang jumlahnya kurang dari 100 dan lebih dari sama dengan 20 kata.

In [14]:
max_review_data_sentence_length = review_data['Num_words_text'].max()

mask = (review_data['Num_words_text']<100) & (review_data['Num_words_text'] >=20)
df_short_reviews = review_data[mask]

Menampilkan hasil yang telah di pisahkan jumlah katanya pada fitur Num_words_text

In [15]:
df_short_reviews

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,have bought several vitality canned food produ...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",this confection that been around centuries lig...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe h...,22
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,wild hair taffy ordered this five pound taffy ...,43
...,...,...,...,...,...,...,...,...,...,...,...
14992,14993,B000EGZ99M,A2T1W9JWKJ2IBI,Lisa,1,1,5,1291766400,Delicious and Nutritious,hard trying other members family healthy glad ...,24
14994,14995,B000EGZ99M,AMP2OWAY2BN0Z,Bonnie Suttle,1,1,4,1282694400,RiceSelect Orzo Whole Wheat Pasta,been using product dishes that call white rice...,43
14996,14997,B000EGZ99M,A1MW2HEG4LF56B,S. Mariconda,3,4,5,1170633600,Absolutely wonderful comfort food!,discovered this local grocery store which only...,39
14997,14998,B000EGZ99M,A1GY0FE07QFFLF,Arlington Cory,0,0,5,1326585600,Whole Wheat Orzo Tasty and Versatile,hard find whole wheat orzo supermarket happy f...,27


Melihat jumlah data sebelum dilakukan cleaning

In [16]:
len(review_data.index)

14999

Melihat jumlah data setelah dilakukan cleaning

In [17]:
len(df_short_reviews.index)

9921

Membuat fungsi remove_stopwrds yang berfungsi untuk melakukan pemecahan pada fitur Text dan menghilangkan kata-kata yang tidak penting dalam bahasa inggris. 

In [18]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

def remove_stopwords(text):
  textArr = text.split(' ')
  rem_text = " ".join([i for i in textArr if i not in stop_words])
  return rem_text

df_short_reviews['Text'] = df_short_reviews['Text'].apply(remove_stopwords)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


Melihat isi data setelah dihilangkan stopwords-nya dan dilakukan splitting data.

In [19]:
df_short_reviews.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text,Num_words_text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,bought several vitality canned food products f...,30
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,product arrived labeled jumbo salted peanutsth...,20
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",confection around centuries light pillowy citr...,52
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,looking secret ingredient robitussin believe f...,22
5,6,B006K2ZZ7K,ADT0SRK1MGOEU,Twoapennything,0,0,4,1342051200,Nice Taffy,wild hair taffy ordered five pound taffy enjoy...,43


Import paket ec_core_web_sm dan membuat fungsi lemmatization, berfungsi untuk membagi pada fitur Text menjadi kata kerja (NOUN) dan kata sifat (ADJECTIVE)

In [20]:
import en_core_web_sm
nlp = en_core_web_sm.load()
def lemmatization(texts, allowed_postags=['NOUN', 'ADJ']):
  output = []
  for sent in texts:
    doc = nlp(sent)
    output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return output

/usr/local/lib/python3.7/dist-packages/spacy/language.py:1899: UserWarning: [W123] Argument disable with value [] is used instead of ['senter'] as specified in the config. Be aware that this might affect other components in your pipeline.
  config_value=config["nlp"][key],


In [21]:
text_list=df_short_reviews['Text'].tolist()
print(text_list[1])
tokenized_reviews = lemmatization(text_list)
print(tokenized_reviews[1])

product arrived labeled jumbo salted peanutsthe peanuts actually small sized unsalted sure error vendor intended represent product jumbo
['product', 'jumbo', 'peanutsthe', 'peanut', 'small', 'sized', 'unsalted', 'sure', 'error', 'vendor', 'product', 'jumbo']


In [22]:
dictionary = corpora.Dictionary(tokenized_reviews)
doc_term_matrix = [dictionary.doc2bow(rev) for rev in tokenized_reviews]

Membuat pemodelan dengan LDA berdasarkan jumlah topik yang diberikan (misalkan disini ditentukan 20 topik)

In [34]:
LDA = gensim.models.ldamodel.LdaModel
lda_model = LDA(corpus=doc_term_matrix, id2word=dictionary, num_topics=20, random_state=100, chunksize=1000, passes=50, iterations=250)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np

Melihat topik yang telah ditentukan sebelumnya

In [35]:
lda_model.print_topics()

[(0,
  '0.055*"product" + 0.041*"store" + 0.041*"price" + 0.034*"good" + 0.033*"great" + 0.030*"amazon" + 0.018*"order" + 0.018*"local" + 0.017*"time" + 0.015*"grocery"'),
 (1,
  '0.026*"bitter" + 0.021*"ground" + 0.018*"work" + 0.016*"huge" + 0.014*"husband" + 0.013*"pod" + 0.013*"problem" + 0.013*"brand" + 0.012*"last" + 0.012*"time"'),
 (2,
  '0.035*"time" + 0.021*"small" + 0.021*"popcorn" + 0.018*"large" + 0.018*"full" + 0.018*"pack" + 0.018*"bag" + 0.017*"variety" + 0.015*"package" + 0.014*"fine"'),
 (3,
  '0.054*"candy" + 0.026*"soft" + 0.019*"hard" + 0.018*"chemical" + 0.013*"feed" + 0.013*"tree" + 0.012*"banana" + 0.012*"licorice" + 0.011*"company" + 0.011*"description"'),
 (4,
  '0.048*"chicken" + 0.041*"pill" + 0.029*"weak" + 0.022*"beef" + 0.021*"orange" + 0.021*"corn" + 0.020*"allergy" + 0.016*"truffle" + 0.016*"skin" + 0.016*"espresso"'),
 (5,
  '0.064*"peanut" + 0.062*"butter" + 0.022*"ghee" + 0.020*"hour" + 0.017*"opinion" + 0.017*"last" + 0.014*"big" + 0.013*"plan" + 0.

Visualilasi daftar topik yang telah ditentukan sebelumnya

In [36]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model, doc_term_matrix, dictionary)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0     -0.112520  0.194884       1        1  13.834257
13    -0.175631 -0.034238       2        1   9.311495
16    -0.205186 -0.042491       3        1   7.584933
11    -0.057036  0.121820       4        1   6.417942
6     -0.107666  0.031278       5        1   6.071799
18    -0.149926 -0.055211       6        1   5.302569
19    -0.137196  0.039180       7        1   5.158844
2      0.115926  0.193870       8        1   4.756112
14     0.081016  0.179973       9        1   4.604044
15    -0.096609 -0.070280      10        1   4.437497
17    -0.004921 -0.172421      11        1   4.137223
7      0.001406  0.027641      12        1   3.960831
8      0.008206 -0.139155      13        1   3.813706
12    -0.033317 -0.038525      14        1   3.781498
1      0.150676  0.121647      15        1   3.390385
9      0.113384 -0.172791      16        1   2.831793
4      0.203019 -0.069874      17        1   2.793999
3      0.120586 -0.062978      18        1   2.729683
10     0.101524 -0.029262      19        1   2.608829
5      0.184264 -0.023068      20        1   2.472564, topic_info=          Term         Freq        Total Category  logprob  loglift
201     coffee  3494.000000  3494.000000  Default  30.0000  30.0000
0         food  1615.000000  1615.000000  Default  29.0000  29.0000
66       treat  1146.000000  1146.000000  Default  28.0000  28.0000
185  chocolate   905.000000   905.000000  Default  27.0000  27.0000
258      store  1075.000000  1075.000000  Default  26.0000  26.0000
..         ...          ...          ...      ...      ...      ...
100       high    34.477642   481.810376  Topic20  -4.9166   1.0627
118     review    32.265188   508.597085  Topic20  -4.9829   0.9422
518       work    30.987415   372.862482  Topic20  -5.0233   1.2123
772       next    29.194121   166.922732  Topic20  -5.0829   1.9564
8         well    30.008507   884.051159  Topic20  -5.0554   0.3169

[1123 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
169       1  0.767072    able
169      12  0.046725    able
169      16  0.097344    able
169      18  0.038938    able
169      20  0.046725    able
...     ...       ...     ...
2037     16  0.985902  yogurt
153       5  0.983653   young
4045      2  0.972203   yuban
1764      3  0.965460    yuck
39        4  0.990794   yummy

[1999 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 14, 17, 12, 7, 19, 20, 3, 15, 16, 18, 8, 9, 13, 2, 10, 5, 4, 11, 6])

In [37]:
print('\nPerplexity: ', lda_model.log_perplexity(doc_term_matrix, total_docs=10000))
from gensim.models.coherencemodel import CoherenceModel
coherence_model_lda = CoherenceModel(model=lda_model, texts=tokenized_reviews, dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Output streaming akan dipotong hingga 5000 baris terakhir.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np


Perplexity:  -7.6316629823013775

Coherence Score:  0.3683066923002187
